## Importación de Librerías y Configuración Inicial

In [3]:
import os
import glob
import json
import re
from collections import Counter

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import nltk
from nltk.corpus import stopwords

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

import spacy
from transformers import pipeline

In [2]:
nltk.download('punkt')
nltk.download('stopwords')

try:
    nlp = spacy.load('es_core_news_sm')
except Exception as e:
    print("No se pudo cargar el modelo de spaCy para español.")

# Se definen las stopwords en español
STOP_WORDS = set(stopwords.words('spanish'))

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Jesus\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Jesus\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Cargar los datos | Preprocesamiento de Datos

### Funciones

In [4]:
def load_json_files(directory_path):
    """Carga todos los archivos JSON"""
    file_pattern = os.path.join(directory_path, "*.json")
    files = glob.glob(file_pattern)
    data = []
    for file in files:
        try:
            with open(file, 'r', encoding='utf-8') as f:
                file_data = json.load(f)
                data.extend(file_data)
            print(f"Archivo cargado: {file}")
        except Exception as e:
            print(f"Error al leer {file}: {e}")
    return data

def clean_text(text):
    """
    Limpia el texto:
      - Elimina etiquetas HTML.
      - Remueve caracteres no alfabéticos (se conservan acentos y ñ).
      - Convierte a minúsculas.
      - Tokeniza y lematiza usando spaCy.
      - Elimina stopwords.
    """
    text = re.sub(r'<.*?>', '', text)  # Eliminar HTML
    text = re.sub(r'[^a-zA-ZáéíóúÁÉÍÓÚñÑ\s]', '', text)  # Solo letras y espacios
    text = text.lower()
    doc = nlp(text)
    tokens = [token.lemma_ for token in doc 
              if token.text not in STOP_WORDS and not token.is_punct and not token.is_space]
    return " ".join(tokens)

def combine_and_clean(row):
    """Combina las columnas 'direct_snippet' y 'full_content' y aplica la limpieza."""
    direct = row.get('direct_snippet', '')
    full = row.get('full_content', '')
    combined = f"{direct} {full}"
    return clean_text(combined)


In [7]:
def extract_pos_tags(text):
    """Extrae las etiquetas POS (part-of-speech) de cada token en el texto usando spaCy"""
    doc = nlp(text)
    return [(token.text, token.pos_) for token in doc]

def extract_entities(text):
    """Extrae las entidades nombradas del texto usando spaCy"""
    doc = nlp(text)
    return [(ent.text, ent.label_) for ent in doc.ents]

In [ ]:
def get_spacy_embeddings(text):
    ""Retorna el vector de la oración procesada con spaCy."""
    doc = nlp(text)
    return doc.vector

### Carga de Datos y Creación del DataFrame


In [ ]:
# Especifica el directorio donde se encuentran tus archivos JSON
data_directory = '../Data'  # Ajusta la ruta según tu estructura

# Cargar datos desde todos los archivos JSON en el directorio
data = load_json_files_from_directory(data_directory)
df = pd.DataFrame(data)
print(f"Total de registros cargados: {df.shape[0]}")

# Visualizar algunas columnas para confirmar la estructura
df.head()

Archivo cargado: ../Data\data_extracted_topic_1_12022025.json
Archivo cargado: ../Data\data_extracted_topic_2_12022025.json
Archivo cargado: ../Data\data_extracted_topic_3_12022025.json
Archivo cargado: ../Data\results_variations.json
Total de registros cargados: 1907


,query,direct_snippet,url,full_title,full_content
0,¿Qué nivel de aprobación tiene actualmente la ...,César Acuña resaltó que la aprobación de la pr...,https://larepublica.pe/sociedad/2024/12/02/nom...,[Error] Web request failed: 403 Client Error: ...,"El gobernador regional de La Libertad, César A..."
1,¿Qué nivel de aprobación tiene actualmente la ...,"La presidenta de la República, Dina Boluarte, ...",https://peru21.pe/politica/dina-boluarte-vuelv...,[Error] Web request failed: 403 Client Error: ...,"La presidenta de la República, Dina Boluarte, ..."
2,¿Qué nivel de aprobación tiene actualmente la ...,La presidenta Dina Boluarte enfrenta una grave...,https://larepublica.pe/politica/2024/12/25/din...,[Error] Web request failed: 403 Client Error: ...,Dina Boluarte enfrenta récords negativos de ap...
3,¿Qué nivel de aprobación tiene actualmente la ...,Dina Boluarte espera que otra mujer sea elegid...,https://www.infobae.com/peru/2025/02/11/dina-b...,Dina Boluarte espera que otra mujer sea elegid...,Dina Boluarte espera que otra mujer sea elegid...
4,¿Qué nivel de aprobación tiene actualmente la ...,"La aprobación de la presidenta de Perú, ... el...",https://actualidad.rt.com/actualidad/530748-ap...,La aprobación de Dina Boluarte cae a mínimos h...,La aprobación de Dina Boluarte cae a mínimos h...


In [6]:
# Combinar las columnas para tener mayor cantidad de oraciones
df['combined_text'] = df['direct_snippet'].fillna('') + " " + df['full_content'].fillna('')

In [7]:
# Aplicar la limpieza al texto combinado
df['clean_text'] = df.apply(combine_and_clean, axis=1)
print("Texto combinado y limpieza completados.")

Texto combinado y limpieza completados.


In [8]:
# Visualizar un ejemplo
print(df[['direct_snippet', 'full_content', 'combined_text', 'clean_text']].head())

                                      direct_snippet  \
0  César Acuña resaltó que la aprobación de la pr...   
1  La presidenta de la República, Dina Boluarte, ...   
2  La presidenta Dina Boluarte enfrenta una grave...   
3  Dina Boluarte espera que otra mujer sea elegid...   
4  La aprobación de la presidenta de Perú, ... el...   

                                        full_content  \
0  El gobernador regional de La Libertad, César A...   
1  La presidenta de la República, Dina Boluarte, ...   
2  Dina Boluarte enfrenta récords negativos de ap...   
3  Dina Boluarte espera que otra mujer sea elegid...   
4  La aprobación de Dina Boluarte cae a mínimos h...   

                                       combined_text  \
0  César Acuña resaltó que la aprobación de la pr...   
1  La presidenta de la República, Dina Boluarte, ...   
2  La presidenta Dina Boluarte enfrenta una grave...   
3  Dina Boluarte espera que otra mujer sea elegid...   
4  La aprobación de la presidenta de Perú, ...

### Representación de Textos con TF-IDF

In [ ]:
# Configurar el vectorizador TF-IDF
vectorizer = TfidfVectorizer(max_features=5000)
X = vectorizer.fit_transform(df['clean_text'])
print("Vectorización TF-IDF completada.")

### Análisis de Sentimientos con VADER

In [ ]:
analyzer = SentimentIntensityAnalyzer()

In [ ]:
df['vader_score'] = df['combined_text'].apply(lambda x: analyzer.polarity_scores(x)['compound'])
df['vader_sentiment'] = df['vader_score'].apply(lambda x: 'positive' if x > 0.2 else ('negative' if x < -0.2 else 'neutral'))
print("Análisis de sentimientos con VADER completado.")

### Análisis de Sentimientos con Hugging Face